In [1]:
import pandas as pd
import numpy as np
import glob
import os

# ================= CONFIGURATION =================
# 1. Path to your 5 CSV files
# Pattern to match your result files (e.g., "survey_results_*.csv")
# INPUT_FILES_PATTERN = '5_newbies/survey_results_*.csv' 
INPUT_FILES_PATTERN = '5_annotators/survey_results_*.csv' 

# 2. 클래스 정의
CATEGORIES = ['Ascending', 'Descending', 'Passing']
# =================================================

def compute_fleiss_kappa(matrix):
    """
    플라이스 카파 계산 함수
    matrix: (Subject 수 x Category 수) 형태의 빈도 행렬
    """
    N, k = matrix.shape  # N: 샘플 수, k: 카테고리 수
    n = np.sum(matrix[0, :])  # n: 평가자 수 (한 행의 합)

    # 1. 각 카테고리의 전체 비율 (P_j)
    p = np.sum(matrix, axis=0) / (N * n)
    
    # 2. 각 샘플의 일치도 (P_i)
    P_i = (np.sum(matrix * matrix, axis=1) - n) / (n * (n - 1))
    
    # 3. 평균 P (Observed Agreement)
    P_bar = np.mean(P_i)
    
    # 4. 우연에 의한 일치도 (Expected Agreement)
    P_e_bar = np.sum(p * p)
    
    # 5. Kappa 계산
    if P_e_bar == 1: return 1.0
    return (P_bar - P_e_bar) / (1 - P_e_bar)

def main():
    # 1. 파일 로드
    files = glob.glob(INPUT_FILES_PATTERN)
    files.sort()
    
    if len(files) < 2:
        print("오류: 최소 2개 이상의 CSV 파일이 필요합니다.")
        return

    print(f"총 {len(files)}개의 결과 파일을 찾았습니다.")
    
    # 2. 데이터 병합 (Survey_ID 기준)
    merged_df = None
    for i, file_path in enumerate(files):
        df = pd.read_csv(file_path)
        # 컬럼명을 평가자별로 변경 (예: Rater_1, Rater_2...)
        df = df.rename(columns={'Selected_Class': f'Rater_{i+1}'})
        
        if merged_df is None:
            merged_df = df
        else:
            merged_df = pd.merge(merged_df, df, on='Survey_ID', how='inner')

    print(f"데이터 병합 완료: {len(merged_df)} 샘플 x {len(files)} 평가자\n")
    
    rater_cols = [c for c in merged_df.columns if c.startswith('Rater_')]
    N = len(merged_df)

    # ==========================================
    # 3. 전체 (Overall) Fleiss' Kappa 계산
    # ==========================================
    print("=== [1] 전체 일치도 (Overall Fleiss' Kappa) ===")
    
    # 전체 카테고리에 대한 빈도 행렬 생성
    matrix_overall = np.zeros((N, len(CATEGORIES)), dtype=int)
    cat_map = {cat: i for i, cat in enumerate(CATEGORIES)}
    
    for row_idx, row in merged_df.iterrows():
        for rater in rater_cols:
            choice = row[rater]
            if choice in cat_map:
                matrix_overall[row_idx, cat_map[choice]] += 1

    overall_kappa = compute_fleiss_kappa(matrix_overall)
    print(f"Overall Kappa: {overall_kappa:.4f}")
    
    # ==========================================
    # 4. 클래스별 (Class-wise) Fleiss' Kappa 계산
    # ==========================================
    print("\n=== [2] 클래스별 일치도 (Class-wise Kappa) ===")
    print(f"{'Class Name':<15} | {'Kappa Score':<12} | {'Interpretation'}")
    print("-" * 50)
    
    for target_class in CATEGORIES:
        # 이진 분류를 위한 빈도 행렬 생성 (Target vs Others)
        # Col 0: Target Class 선택 수
        # Col 1: 그 외 Class 선택 수
        matrix_binary = np.zeros((N, 2), dtype=int)
        
        for row_idx, row in merged_df.iterrows():
            target_count = 0
            other_count = 0
            for rater in rater_cols:
                if row[rater] == target_class:
                    target_count += 1
                else:
                    other_count += 1
            matrix_binary[row_idx, 0] = target_count
            matrix_binary[row_idx, 1] = other_count
            
        # 해당 클래스에 대한 이진 Kappa 계산
        class_kappa = compute_fleiss_kappa(matrix_binary)
        
        # 해석
        if class_kappa < 0.2: interpret = "Slight"
        elif class_kappa < 0.4: interpret = "Fair"
        elif class_kappa < 0.6: interpret = "Moderate"
        elif class_kappa < 0.8: interpret = "Substantial"
        else: interpret = "Almost Perfect"
        
        print(f"{target_class:<15} | {class_kappa:.4f}       | {interpret}")

if __name__ == "__main__":
    main()

총 5개의 결과 파일을 찾았습니다.
데이터 병합 완료: 200 샘플 x 5 평가자

=== [1] 전체 일치도 (Overall Fleiss' Kappa) ===
Overall Kappa: 0.9025

=== [2] 클래스별 일치도 (Class-wise Kappa) ===
Class Name      | Kappa Score  | Interpretation
--------------------------------------------------
Ascending       | 0.9355       | Almost Perfect
Descending      | 0.8840       | Almost Perfect
Passing         | 0.8947       | Almost Perfect
